<h1>ATMOS Version 1.1.0</h1>
The ATMOS calculator, written in Python. It is able to: 
<li>Determine atmos properties for any given altitude</li>
<li>Determine all airspeeds (including Mach) given a speed and altitude</li>
<li>Determine all airspeeds given Mach and altitude</li>



In [216]:
# Python Imports
import pandas as pd
import numpy as np
import math
import csv

<h2>Load the Standard Atmosphere</h2>
Imports standard atmosphere from a file "/instance/standard_atmos.csv".

Parameters are:
<li>Hg:       Geometric Altitude [ft]</li>
<li>H:        Geopotential Altitude [ft]</li>
<li>T:        Temperature [deg_R]</li>
<li>P:        Presure [lb/ft3]</li>
<li>P/P0:     Presure Ratio </li>
<li>pho:      Density [slug/ft3] </li>
<li>pho/pho0: Density Ratio </li>

In [217]:
# Open csv
df_atmos = pd.read_csv('./instance/standard_atmos.csv', comment="#")

print(df_atmos.head(3))


   Hgeo_ft    H_ft   T_degR  P_lb/ft3    P/P0  pho_slug/ft3  pho/pho0
0    -2500 -2500.0  527.584    2300.5  1.0871      0.002547    1.0711
1        0     0.0  518.670    2116.2  1.0000      0.002378    1.0000
2     2500  2500.0  509.756    1931.9  0.9129      0.002209    0.9289


<h2>User Defines the condition to analize</h2>
User selects
<li>Altitude pressure in ft</li>
<li>Speed KEAS, KCAS or KTAS</li>
<li>Speed value in Kts</li>

In [218]:
h_press_user = 60000 # Units ft
analysis_type = "Mach" # "KEAS", "KCAS", "KTAS" or Mach
user_speed = 1.56912    # Units kts

<h2>Calculation of Atmosheric Parameters</h2>
Process flow:
<li>Extract sealevel contants</li>
<li>Test request is "on table"</li>
<li>Interpolate atmos properties to requested altitude</li>
<li>Caculate airspeeds and Mach</li>

In [219]:
# Extract P at sealevel
p_0 = df_atmos.loc[df_atmos["Hgeo_ft"]==0, "P_lb/ft3"].values[0] 
print(p_0)

2116.2


In [220]:
# Test request is "on table"
hgeo_min = df_atmos["Hgeo_ft"].min()
hgeo_max = df_atmos["Hgeo_ft"].max()

# Test user hgeo winthin table
if hgeo_min > h_press_user:
  print("ERROR: Altitude too low out of range")
  exit()
elif hgeo_max < h_press_user:
  print("ERROR: Altitude too high out of range")
  exit()

In [221]:
# Interpolate atmos properties to requested altitude
pTarget        = np.interp(h_press_user, df_atmos["Hgeo_ft"], df_atmos["P_lb/ft3"])
phoRatioTarget = np.interp(h_press_user, df_atmos["Hgeo_ft"], df_atmos["pho/pho0"])
phoTarget      = np.interp(h_press_user, df_atmos["Hgeo_ft"], df_atmos["pho_slug/ft3"])

print(f"For a requested altitude of: {h_press_user} ft")
print(f"Pressure [lb/ft2]: {pTarget}")
print(f"Density [slug/ft3]: {phoTarget}")


For a requested altitude of: 60000 ft
Pressure [lb/ft2]: 151.03
Density [slug/ft3]: 0.000225691


In [222]:
# Calculate Airspeed

def speed_calc (analysis, speed_defined, phoRatioTarget, pTarget, p_0):
  if analysis.upper() == "KEAS":  # Method: ALT AND KEAS
    KEASTarget = speed_defined
    aTarget    = math.sqrt(1.4*pTarget/phoTarget)*0.5924838
    KTASTarget = KEASTarget/math.sqrt(phoRatioTarget)
    MachTarget = KTASTarget/aTarget
    qTarget    = pTarget*((1+0.2*MachTarget**2)**3.5-1)
    KCASTarget = 661.47*math.sqrt(5*((qTarget/p_0+1)**(2/7)-1))

  elif analysis.upper() == "KTAS":  # Method ALT and KTAS
    KTASTarget = speed_defined
    aTarget    = math.sqrt(1.4*pTarget/phoTarget)*0.5924838
    KEASTarget = KTASTarget*math.sqrt(phoRatioTarget)
    MachTarget = KTASTarget/aTarget
    qTarget    = pTarget*((1+0.2*MachTarget**2)**3.5-1)
    KCASTarget = 661.47*math.sqrt(5*((qTarget/p_0+1)**(2/7)-1))

  elif analysis.upper() == "KCAS":  # Method ALT and KCAS    
    KCASTarget = speed_defined
    aTarget = math.sqrt(1.4*pTarget/phoTarget)*0.5924838
    qTarget =  p_0*((0.2*(KCASTarget/661.47)**2+1)**3.5-1)
    MachTarget = math.sqrt(5*((qTarget/pTarget+1)**(2/7)-1))
    KTASTarget = aTarget*MachTarget
    KEASTarget = KTASTarget*math.sqrt(phoRatioTarget)
  
  elif analysis.upper() == "MACH": #Method ALT and MACH
    MachTarget = speed_defined
    aTarget = math.sqrt(1.4*pTarget/phoTarget)*0.5924838
    KTASTarget = aTarget*MachTarget
    KEASTarget = KTASTarget*math.sqrt(phoRatioTarget)
    qTarget    = pTarget*((1+0.2*MachTarget**2)**3.5-1)
    KCASTarget = 661.47*math.sqrt(5*((qTarget/p_0+1)**(2/7)-1))

  else:
    print("Requested analysis not available")
    KTASTarget = np.NaN
    KCASTarget = np.NaN
    KEASTarget = np.NaN
    MachTarget = np.NaN
    qTarget = np.NaN

  df = {
    'KCAS': KCASTarget,
    'KEAS': KEASTarget,
    'KTAS': KTASTarget,
    'Mach': MachTarget,
    'q': qTarget,
  }

  return df

output = speed_calc(
  analysis=analysis_type,
  speed_defined=user_speed,
  phoRatioTarget=phoRatioTarget,
  pTarget=pTarget,
  p_0=p_0,
  )

print(output)

{'KCAS': 356.44645079140406, 'KEAS': 277.20703835885485, 'KTAS': 899.8521252810896, 'Mach': 1.56912, 'q': 462.29347509474144}


<h2>Output</h2>
Prints output to user and appends to log file.

In [223]:


with open('AtmosLog.txt', 'w') as f:  
    w = csv.DictWriter(f, output.keys())
    w.writeheader()
    w.writerow(output)
